# Job classification using GPT API

Idea is to ask the GPT API to cluster jobs and skills based on the data we have about the jobs and companies. This is the next step of the job title clustering algorithm.

The key I'm using for the API is stored in file_path, the model we use now is the gpt-3.5-turbo. 

Please look at the overall documentation for first overview of GPT Analysis : 
 https://docs.google.com/document/d/1_-qUXYqLTFjUDVwBcQJWZ0RL79xxmJkXt8ohUnsWMRU/edit 

You can also find an overview of this gitlab here : 
https://docs.google.com/document/d/1mSD_hJbMuRJ0ff3PAGXtmfnsqaPZ-vroK8Y4Qel2TlA/edit#heading=h.btpkv3i44968

The jupiter notebook is divided into the following parts : 
- Prompts definitions
- Send Request to API and read response basics
- Functions to chain request and generate extended databases
- Cleaning and ploting functions
- Others

The plots are stored in folder \data\lix_scrapper\plots. 

The original file to use is offers.xlsx . 

Functions are also define to remove the non Quantum related jobs and filter out the failed gpt requests.

In [1]:
import openai
import os
import pandas as pd
from tqdm import tqdm
import re
file_path='metadata\openai_api_key.txt'
import plotly.express as px
import plotly.offline as pyo
## Get api_key.
import nbformat
with open(file_path, "r") as file:
    openai.api_key  = file.readline().strip() 
import numpy as np
import matplotlib.pyplot as plt 
import ast 
from collections import Counter


## Define the prompt : 
Note that some prompts have to be filled with data using .format() syntax.

In [2]:
#score_and_category_prompt is the current prompt for the user to choose the category of the job

score_and_category_prompt=""" The job Title is : {} and  here is a job description : {}
The company name is {} and here is the company description : {}

First add to your response  in the form of a mark out of 10 the confidence you have in the job or the company being Quantum Physics or Quantum technologies related.

Now, here is a list of job categories : 
    
    111: "Permanent/tenure track positions (containing assistant professor, associate professor and full professor)",
    112: "Postdoctoral Research Positions",
    113: "PhD Positions",
    114: "Technician and Lab Support in a public/academic institution",
    12: "Teaching, Education and Outreach in a public/academic institution",
    13: "Internships and Studentships in a public/academic institution",
    21: "Internships and Studentships in a private company",
    22: "Teaching, Education and Outreach in a private company",
    231: "Software Engineering",
    232: "Hardware Engineering",
    233: "Technician and Lab Support in a private company",
    31: "IT Support",
    32: "Consultant and Expert Roles",
    33: "Administrative and Office Support",
    34: "Sales and Marketing",
    35: "Management and Leadership",
    36: "Human Resources",
    37: "Finance and Accounting",
    38: "Inventory Control"

Then add to your response the number and name of the category that fit the most the job description and Title."""

skills_prompt=""" I am a Quantum Physics student and I want to identify Qualifications, skills and requirements that are asked from recruiters on their job post so that I can develop those qualifications. Here are a few example of  qualifications but a lot of others could fit : 
Strong coding skills
Self-motivated and curious
Experience with API development
Ability to collaborate effectively with cross-functional teams
Proficiency in SQL and other data modeling tools.
The candidate must have a Ph.D. degree in a relevant field
Strong research skills and a background in computational science and engineering are desired.
Excellent written and verbal communication skills in English and Arabic.

Now please extract the qualifications  that are explicitly required from the following job description : {}
Please return only the list of qualifications and return "Nothing Explicit" if you failed findings qualifications """


skills_cluster_prompt=""" Here are 6 categories of skills :

1) Quantum and physics related skills including atomic physics
2) Non-quantum computer science related skills such as programming
3) Non-quantum enabling technology hardware skills such as laboratory equipment, instrumentation, cryogenic and vacuum systems, optics (this category contains direct references to experimental apparatus whereas more generic methods such as “Experience in experimental design, data acquisition, and analysis.” Would belong in category )
4) Soft/management/business skills such as communication, collaboration, team work and "Strong problem-solving skills and attention to detail." and intellectual property rights and patent law and patent writing
5) Research skills such as "Experience with experimental design, data analysis, and modeling", "Ability to conduct independent research and develop innovative solutions"
6) All others skills : All skills that do not fit in the previous categories

please return an array of the exact size of the number of skills in input and  each value should be the number of the cluster you think the skills is the most appropriate to. return only the array and nothing else. If there is no skills to classify, return an empty array.
Here is the list of skills to be classified : {} """


with open(r'C:\Users\Simonis\Documents\quantum-jobs\competenceframework_clean1.txt','r', encoding='utf-8') as f:
    competences_1=f.read()

competence_prompt = """ Here is a list of competences {}. In this list, the competences are the capitalised headings, such as BASIC QUANTUM CONCEPTS. Each competence is associated with a number. BASIC QUANTUM CONCEPTS is 1.1. Each competence also has keywords associated with it. 

I’m going to give you a list of job requirements. Please identify, if any, the competences involved in each job requirement. Return the competence and the number from the list for which you think there is a match and the requirement which matches the competence. 
For example, here is a list of requirements [‘ 'trapped ions', 'Ability to collaborate effectively with cross-functional teams', ' Experience with quantum gates', 'QED', Experienced FC (Financial Controller)'] The output should be formatted like this:
['trapped ions': 4.3 NEUTRAL ATOMS AND IONS ,'Experience with quantum gates' : 4.6 QUANTUM STATE CONTROL,  'QED' : “2.2 QUANTUM OPTICS AND ELECTRODYNAMICS” , 'Financial Controller' : 8.2 BUSINESS STRATEGY, ENTREPRENEURSHIP AND MANAGEMENT: (BUSINESS LEVEL)]
 
Here are some guidelines to help you classify the skills :
All computer science, development, programming, software engineering, IT skills should be classified in the competence 3.5
All PHD, Masters and Bachelors should be classified in the competence 1.1 if they are in physics, 1.2 if they are in engineering, 3.5 if they are in computer science and so on.
If you find some specific group of words that are also in the list of competences, you can classify them in the competence that fits the best. For instance "quantum sensors" should be classified as 6.something.

Here is the list of skills i want you to classify:
{} 
  """

## Send request to the API and read response

In [3]:
def send_request(prompt):
    #  Send a request to the OpenAI API and return the response
    params = {
        'model': 'gpt-3.5-turbo',
        'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
                     {'role': 'user', 'content': prompt}],  
        'temperature': 0.3,  
        'n': 1, 
    }
     
    response = openai.ChatCompletion.create(**params)

    if response.choices:
        return response.choices[0].message['content']
    else:
        return None

   
categories_dict = {
    111: "Permanent/tenure track positions (containing assistant professor, associate professor and full professor)",
    112: "Postdoctoral Research Positions",
    113: "PhD Positions",
    114: "Technician and Lab Support in a public/academic institution",
    12: "Teaching, Education and Outreach in a public/academic institution",
    13: "Internships and Studentships in a public/academic institution",
    21: "Internships and Studentships in a private company",
    22: "Teaching, Education and Outreach in a private company",
    231: "Software Engineering",
    232: "Hardware Engineering",
    233: "Technician and Lab Support in a private company",
    31: "IT Support",
    32: "Consultant and Expert Roles",
    33: "Administrative and Office Support",
    34: "Sales and Marketing",
    35: "Management and Leadership",
    36: "Human Resources",
    37: "Finance and Accounting",
    38: "Inventory Control"
}

def clean_response(response):
    """ Extract the confidence score and the category from the response of the API
    Input : response of the API
    Output : confidence score out of ten and category name """
    
    # first we extract all numbers from the response :
    numbers_list=re.findall(r'\d+', response)
    if len(numbers_list)==3:
        score=int(numbers_list[0])
        category = categories_dict[int(numbers_list[2])]
        return(score,category)
    if len(numbers_list)==1:
        category = categories_dict[int(numbers_list[0])]
        return(0,category)
    print("Error in the response")
    return("Fail","Fail")

def create_skills_list(file,output_file):
    df=pd.read_excel(file).reset_index(drop=True)
    clean_skills_list=[]
    for skills in df["skills"]:
        clean_skills_list.append(skills.split("\n")[1:])
    df["clean_skills"]=clean_skills_list
    df.to_excel(output_file,index= False)

# create_skills_list(r"C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_skills.xlsx",r"C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\trial_skills.xlsx")

def extract_competence_numbers(file,output_file):
    """ Extract the numbers of the competence from the string of the competence 
      add tà the dataframe a column with the list of numbers of the competence """
    df=pd.read_excel(file)
    df['competence_numbers']=df['competences'].apply(lambda x : re.findall(r'\d+\.\d+', x))
    df.to_excel(output_file,index= False)

# extract_competence_numbers(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_skills_competences.xlsx',r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_skills_competences.xlsx')


## Send chain request 
Idea is to send a request for each job_description on the file.

In [4]:
def create_categories_for_job(file,output_file):
    """ Input : excel file containing job title,description and company name and description
        Output : excel file containing job description, company description and job category thanks to the API of GPT-3"""
    df=pd.read_excel(file).reset_index(drop=True)
    job_titles=df['Title']
    job_descriptions=df['Description']
    company_names=df['Organisation Name']
    company_descriptions=df['overview']

    scores=[]
    categories=[]
    job_responses=[]
    failed_count=0
    misclassified_list=[]
    for i,job_description in enumerate(tqdm(job_descriptions)):
        try : 
            job_description.replace('About the job','').replace('See more','')
            prompt=score_and_category_prompt.format(job_titles[i],job_description,company_names[i],company_descriptions[i])
            category=''
            response=send_request(prompt)
            job_responses.append(response)
            score,category=clean_response(response)
            scores.append(score)
            categories.append(category)
        except Exception as e : 
            job_responses.append(str(e))
            scores.append('Fail')
            categories.append('Fail')
            failed_count+=1
    output_df=df.copy()
    output_df['api_responses']=job_responses
    output_df['Quantum_confidence_score']=scores
    output_df['category']=categories
    print("Number of failed request : " ,failed_count)
    output_df.to_excel(output_file, index=False)

# create_categories_for_job(r'data\lix_scrapper\offers_with_descriptions.xlsx',r'data\lix_scrapper\offers_with_descriptions_gpt2.xlsx') 
       

In [5]:
def create_skills_for_job(file,output_file):
    df=pd.read_excel(file).reset_index(drop=True)
    job_titles=df['Title']
    job_descriptions=df['Description']
    
    requierements=[]
    failed_count=0
    for i,job_description in enumerate(tqdm(job_descriptions)):
        prompt=skills_prompt.format(job_description)
        skills=''
        try : 
            skills=send_request(prompt)
            requierements.append(skills)
        except Exception as e : 
            requierements.append(str(e))
            failed_count+=1

    # output_df=df[['Title','Description','Organisation Name','category']].copy()
    output_df=df.copy()
    output_df['skills']=requierements
    print("Number of failed request : ", failed_count)
    output_df.to_excel(output_file, index=False)
    print('here',output_df.columns)
    create_skills_list(output_file,output_file)

# create_skills_for_job(r'data\lix_scrapper\offers_with_descriptions_gpt_final.xlsx',r'data\lix_scrapper\offers_with_skills.xlsx')

In [6]:
def create_skills_cluster(file,output_file):
    df=pd.read_excel(file).reset_index(drop=True)
    skills=df['clean_skills'].apply(lambda x : eval(x))
    failed_count=0
    clusters=[]
    for i,list_skill in enumerate(tqdm(skills)):
        try : 
            prompt=skills_cluster_prompt.format(list_skill)
            array=send_request(prompt)
            clusters.append(array)
        except Exception as e :
            print(str(e)) 
            clusters.append(str(e))
            failed_count+=1

    output_df=df.copy()
    output_df['clusters']=clusters
    print("Number of failed request : " , failed_count )
    output_df.to_excel(output_file, index=False)

# create_skills_cluster(r'data\lix_scrapper\offers_with_skills.xlsx',r'data\lix_scrapper\offers_with_skills_clusters.xlsx')

In [7]:
def create_competences(file,output_file):
    df=pd.read_excel(file).reset_index(drop=True)
    skills=df['clean_skills'].apply(lambda x : eval(x))
    failed_count=0
    competences=[]
    for i,list_skill in enumerate(tqdm(skills)):
        try :
            if len(list_skill)>=1:
                prompt=competence_prompt.format(competences_1,list_skill)
                response=send_request(prompt)
                competences.append(response)
            else:
                competences.append('Not enough skills')
        except Exception as e :
            print(str(e)) 
            competences.append(str(e))
            failed_count+=1

    output_df=df.copy()
    # output_df=df[['Title','Description','Organisation Name','Quantum_confidence_score','category','clean_skills']].copy()
    output_df['competences']=competences
    print("Number of failed request : " , failed_count )
    output_df.to_excel(output_file, index=False)

# create_competences(r'data\lix_scrapper\offers_with_skills.xlsx',r'data\lix_scrapper\offers_with_skills_competences.xlsx')

## Cleaning And Ploting 

In [24]:
def clean_database(file,output_file,threshold=2):
    """ Input file is database with quantum confidence score and category resulting from the create_category_from_job function
        Output : Filtered database with only jobs that have a quantum confidence score above a certain threshold """
    df=pd.read_excel(file)
    df=df[df['Quantum_confidence_score'].apply(lambda x : x =='Fail' or x>=threshold)]
    df.to_excel(output_file, index=False)

# clean_database(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_descriptions_gpt_final.xlsx',r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_descriptions_gpt_final.xlsx')

colors = {
    "Permanent/tenure track positions (containing assistant professor, associate professor and full professor)": 'cornflowerblue',
    "Technician and Lab Support in a public/academic institution": 'midnightblue',
    "Postdoctoral Research Positions": 'royalblue',
    "PhD Positions": 'steelblue',
    "Teaching, Education and Outreach in a public/academic institution": 'skyblue',
    "Internships and Studentships in a public/academic institution": 'lightblue',
    "Internships and Studentships in a private company": 'indianred',
    "Teaching, Education and Outreach in a private company": 'lightcoral',
    "Software Engineering": 'firebrick',
    "Hardware Engineering": 'salmon',
    "Technician and Lab Support in a private company": 'darkred',
    "IT Support": 'mediumseagreen',
    "Consultant and Expert Roles": 'palegreen',
    "Administrative and Office Support": 'darkseagreen',
    "Sales and Marketing": 'limegreen',
    "Management and Leadership": 'darkolivegreen',
    "Human Resources": 'olivedrab',
    "Finance and Accounting": 'darkgreen',
    "Inventory Control": 'mediumaquamarine',
    "Fail": 'black'
}


category_order = list(colors.keys())

def create_pie_categories(file,zone:str='Worlwide',save=False,plot=True):
    """ Input: database with quantum confidence score and category resulting from the create_category_from_job function
        Output: Pie chart of the distribution of the categories """
    
    df = pd.read_excel(file)
    values_count = df['category'].value_counts()
    ordered_values_count = values_count.reindex(category_order)

    #drop empty categories in the color map and the dataframe, also drop the failed request
    nan_indices = df[df.isna()].stack().index.tolist()
    for index in nan_indices:
        key = df.loc[index].values[0]
        if key in colors:
            del colors[key]
    ordered_values_count.dropna(inplace=True)
    ordered_values_count.drop(index=['Fail'],inplace=True)
    fig, ax = plt.subplots(figsize=(30,15))
    ax.pie(ordered_values_count, labels=ordered_values_count.index, colors=[colors[label] for label in ordered_values_count.index],autopct='%1.1f%%')
    ax.set_title('Distribution of the categories {}'.format(zone),fontweight='bold')
    ax.axis('equal') 
    if save: 
        plt.savefig(r'data\lix_scrapper\plots\Distribution_of_categories_{}.png'.format(zone))
    if plot:
        plt.show()



def create_global_competence_hist(file,plot=False,save=False,zone:str='Worlwide'):
    """ Input : job database with competence number (extracted from the create_competences function and then Extract_competence_numbers function)
        Output : Horizontal Histogram of the distribution of the Domain of competence
    """
    df=pd.read_excel(file)
    df=df[df['competence_numbers'].apply(lambda x : x!='[]')]
    list_competences=[number for number in df['competence_numbers'].apply(lambda x : eval(x))]
    list_domains=[number[j][0] for number in list_competences for j in range(len(number))]
    try:
        list_domains.remove('0')
    except:
        None
    counter=Counter(list_domains)
    domains=(list(counter.keys()))
    counts=(list(counter.values()))
    domains, counts = zip(*sorted(zip(domains, counts)))
    plt.figure(figsize=(20,10))
    labels=['1: Concepts and Foundations', '2: Physical Foundations of Quantum Technologies', '3: Enabling Technologies and techniques', '4: Quantum Hardware', '5: Quantum Computing and Simulation', '6: Quantum Sensors and imaging Systems', '7: Quantum Communication and Networks', "8: Valorisation"]
    plt.barh(domains,counts,align='center')
    plt.title('Distribution of the domains of competences ({})'.format(zone),fontweight='bold')
    plt.yticks(range(len(labels)),labels)
    plt.ylabel('Domain of competence')
    plt.xlabel('Number of jobs')
    if save:
        plt.tight_layout()
        plt.savefig(r'data\lix_scrapper\plots\Distribution_of_domains_competences_{}.png'.format(zone))
    if plot:
        plt.show()
    
def create_domain_specific_hist(file,plot=False,save=False,zone:str='Worlwide'):
    """ Input:job database with competence number (extracted from the create_competences function and then Extract_competence_numbers function)
        Action : For each of the 8 Domains, create a histogram with the distribution of the subdomains
    """
    df=pd.read_excel(file)
    df=df[df['competence_numbers'].apply(lambda x : x!='[]')]
    list_competences=[number for number in df['competence_numbers'].apply(lambda x : eval(x))]
    for i in range(1,9):
        domain_competences_list=[number[j] for number in list_competences for j in range(len(number))]
        domain_competences_list=[number for number in domain_competences_list if number[0]==str(i)]
        domain_competences_list=[number[2] for number in domain_competences_list]
        counter=Counter(domain_competences_list)
        domains=(list(counter.keys()))
        counts=(list(counter.values()))
        domains, counts = zip(*sorted(zip(domains, counts))) # sort the values
        plt.bar(domains,counts,align='center')
        plt.title('Distribution of the subdomains of domain {} ({})'.format(i,zone),fontweight='bold')
        plt.xlabel('Subdomain of competence')
        plt.ylabel('Number of jobs')
        plt.xticks(range(len(domains)), [str(i) + '.' + str(j) for j in domains])
        if save:
            plt.savefig(r'data\lix_scrapper\plots\Distribution_of_subdomains_domain_{}_{}.png'.format(i,zone))
        if plot: 
            plt.show()


In [18]:
def merge_with_final_database(file :str,database_path:str=r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_gpt_analysis.xlsx') -> None : 
    """ Input : file path to the new offers
        Action : merge the new offers with the final database
        Also edit the Europe_only File
    """
    df_new_offers=pd.read_excel(file)
    df_final_database=pd.read_excel(database_path)
    df_final_database=pd.concat([df_final_database,df_new_offers],ignore_index=True)
    df_final_database.to_excel(database_path,index=False)
    df=df_final_database[df_final_database['continent']=='Europe']
    df.to_excel('europe_only.xlsx', index=False)
    

# merge_with_final_database(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\new_offers.xlsx')

In [10]:
def gpt_pipeline(file:str='',output_file:str='',show_state:bool=True):
    """
    Input : Excel file generated by LIX pipeline
    Action : Perform all gpt tasks on the file , and merge it with the final database
    """

    #Extract  GPT job Categories
    create_categories_for_job(file,file)
    if show_state :
        print("Job categories created")

    # Remove jobs with quantum confidence score =<2
    clean_database(file,file)

    #Extract GPT skills in job description
    create_skills_for_job(file,file)
    if show_state :
        print("Skills extracted")
    
    #Extract GPT skills clusters
    create_skills_cluster(file,file)
    if show_state :
        print("Skills clusters extracted")

    #Extract GPT competences
    create_competences(file,file)
    if show_state :  
        print("Competences extracted")

    #Extract competence Number
    extract_competence_numbers(file,file)

    #Merge with final database
    merge_with_final_database(file)

# pipeline(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\new_offers.xlsx')

Job categories created
Skills extracted


  0%|          | 2/409 [00:31<2:04:22, 18.33s/it]

The server is overloaded or not ready yet.


 25%|██▍       | 101/409 [09:02<8:02:10, 93.93s/it]

Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Mon, 10 Jul 2023 10:26:03 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e481398ea6c0b83-AMS', 'alt-svc': 'h3=":443"; ma=86400'}


...

The server is overloaded or not ready yet.


 84%|████████▎ | 342/409 [19:28<55:14, 49.47s/it]

Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Mon, 10 Jul 2023 10:55:22 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e483e833e1d0b83-AMS', 'alt-svc': 'h3=":443"; ma=86400'}


100%|██████████| 409/409 [22:25<00:00,  3.29s/it]


Number of failed request :  3
Competences extracted


In [ ]:
def plot_skills_clusters_histogram(file,save=False,plot=True,save_clean_file=False):
    """Input : database with the clusters the skills extracted and cleaned from the job description, and with a colunm where there is an array matching the skills to the clusters
        Output : histogram of the distribution of the skills in the cluster
        DEPRECEATED  as this prompt is not used anymore but we might come back to it.
        If you want to make the plot only for some job categories, change the list l in the function
    """

    df=pd.read_excel(file)
    # print(df['category'].unique())

    # l=['PhD Positions','Postdoctoral Research Positions']
    # df=df[df['category'].isin(l)]

    df=df[df['clusters'].apply(lambda x : x[0]=='[' and x[-1]==']')]

    if save_clean_file:
        df.to_excel(file[:-5]+'_clean.xlsx',index=False)

    clusters=df['clusters'].apply(lambda x : np.array(x).flatten()[0]).reset_index(drop=True)
    total_clusters=[]
    for cluster in clusters:
        try : 
            cluster=[int(element) for element in ast.literal_eval(cluster)]
            for nb in cluster:
                total_clusters.append(nb)
        except:
            None

    plt.figure(figsize=(30,15))
    n,bins,patches=plt.hist(total_clusters,bins=6,edgecolor='black')

    labels_long=['1) Quantum and physics related skills including atomic physics','2) Non-quantum computer science related skills such as programming','3) Non-quantum enabling technology hardware skills such as laboratory equipment,',\
                '4) Soft/management/business skills such as communication, collaboration, team work','5) Research skills','6) All others skills'\
                    ,' 6 : Quantum Consultant and Project Manager']
    labels=['Cluster ' + str(i) for i in range(1,7)]

    plt.tick_params(bottom=False)
    plt.xlabel('Cluster')
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    plt.legend(patches,labels_long)
    plt.xticks(bin_centers,labels)
    plt.ylabel('Number of skills matching the cluster')
    plt.title('Histogram of skills appearance in Cluster')

    if save:
        plt.savefig('Histogram of skills appearance in Cluster.png')
    if plot:
        plt.show()

plot_skills_clusters_histogram(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_skills_clusters.xlsx',save=False,plot=True,save_clean_file=False)

## Make plots ! 

In [ ]:
def make_plots():
    create_pie_categories(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_gpt_analysis.xlsx',plot=True,save=True)
    create_global_competence_hist(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_gpt_analysis.xlsx',plot=True,save=True)
    create_domain_specific_hist(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_gpt_analysis.xlsx',plot=True,save=True)
    create_pie_categories(r'C:\Users\Simonis\Documents\quantum-jobs\europe_only.xlsx',zone='Europe',plot=True,save=True)
    create_global_competence_hist(r'C:\Users\Simonis\Documents\quantum-jobs\europe_only.xlsx',plot=True,save=True,zone='Europe')
    create_domain_specific_hist(r'C:\Users\Simonis\Documents\quantum-jobs\europe_only.xlsx',plot=True,save=True,zone='Europe')

# make_plots()

### Put skills extracted in a file 
Was made to show to our partner first results or to help with data verification (manual job) by creating excel regrouping the gpt output per categories.

In [ ]:
df=pd.read_excel(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_skills_clusters.xlsx')
df=df[df['clusters'].apply(lambda x : x[0]=='[' and x[-1]==']')]
skills=df['clean_skills'].apply(lambda x : np.array(x).flatten()[0]).reset_index(drop=True)
list_clusters=df['clusters']
# clusters=df['clusters'].apply(lambda x : np.array(x).flatten()[0]).reset_index(drop=True)
dict_clusters={1:[],2:[],3:[],4:[],5:[],6:[]}
for i,clusters in enumerate(list_clusters):
    try : 
        clusters=[int(element) for element in ast.literal_eval(clusters)]
        # print(clusters)
        # print(type(eval(skills[i])),eval(skills[i]))
        for j,cluster in enumerate(clusters):
            # print(eval(skills[i])[j])
            dict_clusters[cluster].append(eval(skills[i])[j])
    except:
        None

print(len(list(dict_clusters.items())[0][1]))
for cluster in dict_clusters.keys():
    print("Cluster", cluster, "   Number of skills : ", len(dict_clusters[cluster]))
    print(dict_clusters[cluster])
    print('\n'*4)



In [14]:
from openpyxl import Workbook

workbook = Workbook()

for sheet_name, value_list in dict_clusters.items():
    sheet = workbook.create_sheet(title=str(sheet_name))
    for index, element in enumerate(value_list, start=1):
        sheet.cell(row=index, column=1, value=element[2:])

workbook.remove(workbook["Sheet"])

# workbook.save(r"C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\skills_clusters.xlsx")

In [47]:
import pandas as pd 
import re
# This cell is used to create the excel file with the competences  grouped by their numbers 
dic={}
df=pd.read_excel(r'C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\offers_with_gpt_analysis.xlsx')
df=df[df['competences'].apply(lambda x :  x[0] == '[' and x[1]!=']')]
competences_list=df['competences'].apply(lambda x : x[1:-1].replace("[","").replace("]\n",", '").replace("\n'",", '").replace('\n"',""))
fail_count=0
for i,competences in enumerate(competences_list):
    for competence in competences.split(", '"):
        try:
            skill,competence_numbers=competence.split(':')
            numbers=re.findall(r'\d+\.\d+', competence_numbers)
            for number in numbers:
                if number in dic:
                    dic[number].append(skill.replace("'","").replace('\n',''))
                else:
                    dic[number]=[skill.replace('\n','')]
        except Exception as e:
            fail_count+=1
            continue
print(fail_count)
print(dic)


from openpyxl import Workbook

workbook = Workbook()

for sheet_name, value_list in dic.items():
    sheet = workbook.create_sheet(title=str(sheet_name))
    for index, element in enumerate(value_list, start=1):
        sheet.cell(row=index, column=1, value=element[:])

workbook.remove(workbook["Sheet"])
workbook.save(r"C:\Users\Simonis\Documents\quantum-jobs\data\lix_scrapper\competences_validation.xlsx")


    

327
{'1.1': ["- Must have a Ph.D. degree in a relevant field'", 'Doctorate in Physics or other applicable area, or PhD candidate with scheduled defense ', 'PhD in computer science, physics, Quantum, or a related field (preferred)', 'PhD in computer science, physics, Quantum, or a related field (preferred)', 'PhD in computer science, physics, Quantum, or a related field (preferred)', 'Doctorate in Physics or other applicable area, or PhD candidate with scheduled defense', 'Hold a Ph.D. or equivalent in a relevant field by date of hire', 'Hold a Ph.D. or equivalent in a relevant field by date of hire', 'Hold a Ph.D. or equivalent in a relevant field by date of hire ', '- Ph.D. degree in a relevant field is preferred but not explicitly required. ', '- Ph.D. degree in a relevant field (desired) ', 'Strong problem-solving skills', 'Background in physics or computer science', '- Knowledge of Quantum Computing is preferred', 'Ph.D. in Physics, Engineering, or a related field', 'Strong researc